<html dir='rtl'>
<h2 dir='rtl'>
توضیخ کلی کار
</h2>
<p dir='rtl'>
در این نوت بوک،‌ مقاله‌هایی در زمینه مدل‌رانه، توصیه‌گر و استفاده از توصیه‌گر در فرایند مدل‌رانه مورد پردازش قرار گرفته است تا تراکم کلمات استفاده شده در آن و کلماتی که به یکدیگر مرتبط هستند، مشخص شوند.
</p>

<h2 dir='rtl'>
نوت بوک به تفکیک بخش‌ها
</h2>
<p dir='rtl'>
در ادامه بخش‌های مختلف نوت بوک به همراه توضیحاتی درمورد هر بخش، آورده شده است.
</p>
<h3 dir='rtl'>
نصب پیش‌نیازها
</h3>
<p dir='rtl'>
در اولین سلول این نوت بوک، پیش‌نیازهایی که برای اجرای این نوت بوک مورد نیاز است آورده شده است تا نصب شوند. بنابراین این سلول باید قبل از اجرای سلول‌های دیگر اجرا شود.
</p>

<h3 dir='rtl'>
درون‌ریزی کتابخانه‌ها و تنظیمات اولیه
</h3>
<p dir='rtl'>
در این بخش، کتابخانه‌هایی که در ادامه مورد نیاز بوده اند درون‌ریزی شده اند و برخی کتابخانه‌ها که نیاز به پیش‌تنظیمات داشته اند، تنظیم شده اند.
</p>

<h3 dir='rtl'>
خزنده
</h3>
<p dir='rtl'>
در این بخش، خزنده کلیه‌ی فایل‌های پی دی اف را پردازش کرده و متن آنها را از صفحات مختلف استخراح می‌کند تا در مراحل بعد پیش‌پردازش و پردازش روی متن استخراج شده انجام گیرد.
</p>

<h3 dir='rtl'>
ابزار تمیزکردن متن و جملات
</h3>
<p dir='rtl'>
در این بخش ابزارهایی به منظور بالابردن خوانایی در پیش‌پردازش تعریف شده اند که یکی برای تمیز کردن متنی است که به صورت خام استخراج شده است و دیگری برای تمیز کردن جملات.
</p>

<h3 dir='rtl'>
خواندن دیتاست و پیش‌پردازش آن
</h3>
<p dir='rtl'>
در این گام دیتاستی که ذخیره شده بود، خوانده می‌شود و سپس مراحل پیش‌پردازش به کمک ابزارهای تعریف‌شده در بخش قبل و ابزارهای کتابخانه‌های درون‌ریزی شده مانند nltk، انجام می‌شوند.
</p>

<h3 dir='rtl'>
ابزار پردازش وردنت
</h3>
<p dir='rtl'>
برای بهتر شدن خوانایی کد،‌ دو تابع تعریف شده اند که به ترتیب بسامد کلمات و ارتباط آنها با یکدیگر را توسط وردنت تحلیل می‌کند.
</p>

<h3 dir='rtl'>
پردازش آماری
</h3>
<p dir='rtl'>
در این بخش، آماری از کل جملات مقالات آورده شده است که از جمله‌ی آن تراکم کلمات در جملات و ارتباط بین کلمات شبیه به یکدیگر و گروه‌بندی آنها هستند.
</p>
</html>

In [ ]:
# Install Dependencies

%pip install pypdf2
%pip install pycryptodome
%pip install nltk
%pip install tqdm
%pip install spacy
!python -m spacy download en_core_web_sm

In [3]:
# Import required modules and initial config

import PyPDF2
import glob
import pickle
import string
import nltk
import re
import spacy
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from string import *
from nltk.probability import FreqDist
import numpy as np
import itertools
import re
from nltk.corpus import wordnet as wn
from collections import defaultdict

DATASEST_FILENAME = 'dataset.pickle'
nlp = spacy.load('en_core_web_sm') # or whatever model you have installed
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Crawl all text of the articles

all_text = []

for f in glob.glob("corpus/*.pdf"):
    
    with open(f, 'rb') as f_obj:
        file_sentences = []
        
        # creating a pdf reader object
        pdfReader = PyPDF2.PdfFileReader(f_obj)
  
        # printing number of pages in pdf file
        for p in range(pdfReader.numPages):
            # creating a page object
            pageObj = pdfReader.getPage(p)
        
            # extracting text from page
            file_sentences.append(pageObj.extractText())
            
        all_text.append(file_sentences)
        
with open(DATASEST_FILENAME, 'wb+') as f:
    pickle.dump(all_text, f)
    

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


In [6]:
# Sanitizing text and sentences utils

def sanitize_text(text):
    # try to remove all refrencese 
    text = re.sub(r'\n\[\s*\d\s*\][^\n]+\n', r'\n', text)
    
    # remove links
    text = re.sub(r'http[\S]+[\s]', '', text)
    text = re.sub(r'(www|WWW)[\S]+[\s]', '', text)
    
    # remove spacee before the dash in names having dash
    text = re.sub(r'[ ](-\w)', r'\1', text)

    # complete sentences splited by breakline
    text = re.sub(r'-[\n]', '', text)

    # remove non-content setntences
    text = re.sub(r'[\n].+[^.][\n]', '\n', text)

    # replace multispace with single space
    text = re.sub(r'[\s]+', ' ', text)

    # remove references
    text = re.sub(r'\[.+?\]', '', text)
    
    # remove three dots
    text = re.sub(r'[.]{3} | [*]', '', text)
        
    return text

def normalize_sentence(tokenized_sents, minimum_length=2, stopword_removal=True, stopwords_domain=[], lower_case=False, punctuation_removal=True):
    '''
    normalization function
    '''
    normalized_sents = tokenized_sents
    
    if stopword_removal:
        # Remove stopwords in English and also the given domain stopwords
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        normalized_sents=[[word for word in sentence if (word.lower() not in stopwords_domain + stopwords)] for sentence in tokenized_sents ]

    if punctuation_removal:
        # Remove punctuations
        normalized_sents=[[word for word in sentence if word not in string.punctuation] for sentence in normalized_sents ]

    if lower_case:
        # Convert everything to lowercase and filter based on a min length
        normalized_sents=[[word.lower() for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]

    elif minimum_length>1:
        normalized_sents= [[word for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]        
        
    return normalized_sents

In [7]:
# Read dataset and pre-prcoessing

with open(DATASEST_FILENAME, 'rb') as f:
    all_text = pickle.load(f)

# merge all pages of each article
articles = ['\n'.join(article_pages_list) for article_pages_list in all_text]

# merge all articles
text = '\n'.join(articles)
text = sanitize_text(text)
sentences = nltk.tokenize.sent_tokenize(text)
tokenized_sentences = [[word for word in nltk.tokenize.word_tokenize(sent) if
                        len(word)<30 and word != '...'] for sent in sentences]
tokenized_sentences = filter(lambda x: len(x) > 6, tokenized_sentences)
normalized_sentences = normalize_sentence(tokenized_sentences)

In [ ]:
# WordNet util functions

def categories_from_hypernyms(termlist):
    hypterms = []
    hypterms_dict = defaultdict()
    for term in termlist:                  # for each term
        s = wn.synsets(term.lower(), 'n')  # get its nominal synsets
        for syn in s:                      # for each lemma synset
            for hyp in syn.hypernyms():    # It has a list of hypernyms
                hypterms.append(hyp.name())      # Extract the hypernym name and add to list
                if hyp.name() not in hypterms_dict:
                    hypterms_dict[hyp.name()] = list()
                hypterms_dict[hyp.name()].append(term)  # Extract examples and add them to dict
                
    hypfd = nltk.FreqDist(hypterms)             # After going through all the nouns, print out the hypernyms 
    for (name, count) in hypfd.most_common(25):  # that have accumulated the most counts (have seen the most descendents)
        print( name, '({0})'.format(count))
        print ('\t', ', '.join(set(hypterms_dict[name])))  # show the children found for each hypernym
        print ()
        
def freq_normed_unigrams(sents):
    wnl = WordNetLemmatizer()
    
    tagged_POS_sents = [nltk.pos_tag(sent) for sent in sents]
    
    normed_tagged_words = [wnl.lemmatize(word[0].lower()) for sent in tagged_POS_sents
                           for word in sent 
                           if word[0].lower() not in nltk.corpus.stopwords.words('english')
                           and word[0] not in punctuation
                           and not re.search(r'''^[\.,;"'?!():\-_`]+$''', word[0])
                           and word[1].startswith('N')]

    top_normed_unigrams = [word for (word, count) in nltk.FreqDist(normed_tagged_words).most_common(40)]
    return top_normed_unigrams

In [20]:
# Generate different statastics

# frequency statastics
mp_freqdist = FreqDist(itertools.chain(*normalized_sentences))
top20words=mp_freqdist.most_common(20)
print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
for topword in top20words:
    percent=(topword[1]/len(normalized_sentences))*100
    print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

print('\n', 60*'-', '\n')

# word and snetence averages
sentence_words = list(itertools.chain(*normalized_sentences))
print ('%-16s' % 'Number of words', '%-16s' % len(sentence_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(sentence_words)))
avg=np.sum([len(word) for word in sentence_words])/len(sentence_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
avg_sentence_length_c=np.mean([len(' '.join(sentence)) for sentence in normalized_sentences])
print ('%-16s' % 'Average sentence length in characters', '%-16s' % avg_sentence_length_c)
print ('%-16s' % 'Longest word', '%-16s' % sentence_words[np.argmax([len(word) for word in sentence_words])])

# WordNet
top_words = freq_normed_unigrams(normalized_sentences)
categories_from_hypernyms(top_words)


person.n.01 (4)
	 user, case

activity.n.01 (4)
	 support, process, use

body_part.n.01 (3)
	 process, feature, system

collection.n.01 (3)
	 class, data, information

cognition.n.01 (3)
	 process, information

concept.n.01 (3)
	 section, rule

direction.n.06 (3)
	 rule

hypothesis.n.02 (2)
	 model, framework

assistant.n.01 (2)
	 model

ideal.n.01 (2)
	 model, example

representation.n.01 (2)
	 model, example

method.n.01 (2)
	 technique, system

change.n.01 (2)
	 transformation, development

change_of_integrity.n.01 (2)
	 transformation

information.n.02 (2)
	 example, data

part.n.02 (2)
	 element, section

substance.n.01 (2)
	 element

use.n.01 (2)
	 application, development

part.n.01 (2)
	 item

fact.n.01 (2)
	 item, case

supporting_structure.n.01 (2)
	 support, framework

happening.n.01 (2)
	 example, case

gathering.n.01 (2)
	 class

time_period.n.01 (2)
	 time

resource.n.01 (2)
	 support

